In [ ]:
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import dash_ag_grid as dag
import pandas as pd
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

In [2]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/XueliZhu/CS-340/refs/heads/main/aac_shelter_outcomes.csv')
df.rename(columns={df.columns[0]:'index'}, inplace=True)
columns = [{'headerName':col, 'field':col} for col in df.columns]

In [4]:
filter_function = {
    'Wate_Rescue': (
        "params.data.age_upon_outcome_in_weeks >= 26  && "
        "params.data.age_upon_outcome_in_weeks <= 156 && "
        "(params.data.breed == 'Labrador Retriever Mix' || "
        "params.data.breed == 'Chesapeake Bay Retriever' || "
        "params.data.breed == 'Newfoundland') && "
        "params.data.sex_upon_outcome == 'Intact Female' "
        ),
    'Mountain_or_Wilderness': (
        "params.data.age_upon_outcome_in_weeks >= 26 && "
        "params.data.age_upon_outcome_in_weeks <= 156 && "
        "(params.data.breed == 'German Shepherd' || "
        "params.data.breed == 'Alaskan Malamute' || "
        "params.data.breed == 'Old English Sheepdog' || "
        "params.data.breed == 'Siberian Husky' || "
        "params.data.breed == 'Rottweiler' ) && "
        "params.data.sex_upon_outcome == 'Intact Male' "
    ),
    'Disaster_or_Individual': (
        "params.data.age_upon_outcome_in_weeks >= 20  && "
        "params.data.age_upon_outcome_in_weeks <= 300  && "
        "(params.data.breed == 'Doberman Pinscher' || "
        "params.data.breed == 'German Shepherd' || "
        "params.data.breed == 'Golden Retriever' || "
        "params.data.breed == 'Bloodhound' || "
        "params.data.breed == 'Rottweiler' ) && "
        "params.data.sex_upon_outcome == 'Intact Male' "
    ),
    'Reset': "true"
}

In [5]:
app.layout = dbc.Container([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(
        html.Center(
            html.Img(
                src='https://i.ibb.co/y0DgFXY/Grazioso-Salvare-Logo.png',
                style={'width':'18%', 'height':'auto'})),
        href='https://www.snhu.edu',
        target='blank'
        ),
    html.Center(html.B(html.H1('Welcome to Grazioso Salvare Dashboard', 
                               style={
                                   'color':'rgb(201, 19, 75)',
                                   'fontFamily':'Times New Roman'
                               }))),
    html.Center(html.H6('Developed by Xueli Zhu',style={'fontFamily':'Times New Roman'})),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='external-filter-radio',
            options={
                'Wate_Rescue': 'Water Rescue',
                'Mountain_or_Wilderness': 'Mountain or Wilderness',
                'Disaster_or_Individual': 'Disaster or Individual',
                'Reset': '  Reset',
            },
            inline=True,
            labelStyle={'margin-right':'20px'},
            style={'margin': '10px'}
        )
    ]),
    dag.AgGrid(
            id="dashGrid",
            columnDefs=columns,
            rowData=df.to_dict("records"),
            defaultColDef={'flex':0},
            columnSize='autoSize',
            dashGridOptions={
                "domLayout":"normal",
                "rowSelection": "single",
                "animateRows": False,
                "pagination":True,
                "paginationPageSize": 10,
                },
                style={
                    'overflowX':'auto',
                    'overflowY':'auto',
                    'height':260,
                    'width':'100%',
                    },
        ),
    html.Pre(id='selected-row', style={'text-wrap':'wrap'}),
    dbc.Row([
        dbc.Col(dcc.Graph(id='pie-chart'), width=6),
        dbc.Col(html.Div(id='map-id', className='col s12 m6'), width=6),
    ])
])

In [6]:
@app.callback(
    Output("pie-chart", "figure"),
    Input("external-filter-radio", "value"),
    prevent_initial_call=False,
)
def update_external_filter(filter_value):
    if filter_value is None or filter_value == 'Reset':
        top_10_breeds = df['breed'].value_counts().head(10).reset_index()
        top_10_breeds.columns = ['breed', 'count']
        fig = go.Figure(
            data=[
                go.Pie(labels=top_10_breeds['breed'], 
                       values=top_10_breeds['count'], 
                       marker=dict(
                           colors = ["rgb(201, 19, 75)", "rgb(206, 42, 92)", "rgb(219, 69, 115)", "rgb(227, 98, 138)", "rgb(229, 119, 153)", "rgb(233, 143, 170)", "rgb(236, 162, 185)", "rgb(239, 185, 201)", "rgb(243, 201, 214)", "rgb(231, 207, 215)"]))])
        fig.update_layout(title="Top 10 Breeds in Data")
        return fig
    elif filter_value == 'Wate_Rescue':
        filtered_df= df.query("age_upon_outcome_in_weeks >= 26 and age_upon_outcome_in_weeks <= 156 and breed in ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'] and sex_upon_outcome == 'Intact Female'")
        breed_counts = filtered_df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        fig = go.Figure(
            data=[
                go.Pie(labels=breed_counts['breed'], 
                       values=breed_counts['count'], 
                       marker=dict(
                           colors = ["rgb(201, 19, 75)", "rgb(206, 42, 92)", "rgb(219, 69, 115)", "rgb(227, 98, 138)", "rgb(229, 119, 153)", "rgb(233, 143, 170)", "rgb(236, 162, 185)", "rgb(239, 185, 201)", "rgb(243, 201, 214)", "rgb(231, 207, 215)"]))])
        fig.update_layout(title=f"Breeds for Wate Rescue")
        return fig
    elif filter_value == 'Mountain_or_Wilderness':
        filtered_df= df.query("age_upon_outcome_in_weeks >= 26 and age_upon_outcome_in_weeks <= 156 and breed in ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'] and sex_upon_outcome == 'Intact Male'")
        breed_counts = filtered_df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        fig = go.Figure(
            data=[
                go.Pie(labels=breed_counts['breed'], 
                       values=breed_counts['count'], 
                       marker=dict(
                           colors = ["rgb(201, 19, 75)", "rgb(206, 42, 92)", "rgb(219, 69, 115)", "rgb(227, 98, 138)", 
                                     "rgb(229, 119, 153)", "rgb(233, 143, 170)", "rgb(236, 162, 185)", "rgb(239, 185, 201)", 
                                     "rgb(243, 201, 214)", "rgb(231, 207, 215)"]))])
        fig.update_layout(title=f"Breeds for Mountain or Wilderness")
        return fig
    elif filter_value == 'Disaster_or_Individual':   
        filtered_df= df.query("age_upon_outcome_in_weeks >= 20 and age_upon_outcome_in_weeks <= 300 and breed in ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'] and sex_upon_outcome == 'Intact Male'")
        breed_counts = filtered_df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        fig = go.Figure(
            data=[
                go.Pie(labels=breed_counts['breed'], 
                       values=breed_counts['count'], 
                       marker=dict(
                           colors = ["rgb(201, 19, 75)", "rgb(206, 42, 92)", "rgb(219, 69, 115)", "rgb(227, 98, 138)", "rgb(229, 119, 153)", "rgb(233, 143, 170)", "rgb(236, 162, 185)", "rgb(239, 185, 201)", "rgb(243, 201, 214)", "rgb(231, 207, 215)"]))])
        fig.update_layout(title=f"Breeds for Disaster or Individual")
        return fig

In [7]:
@app.callback(
     Output("dashGrid", "dashGridOptions"),
     Input("external-filter-radio", "value"),
     prevent_initial_call=True,
    )
def update_external_filter(filter_value):
    return {
        "isExternalFilterPresent": {"function": "true" if filter_value != 'Reset' else "false"},
        "doesExternalFilterPass": {"function": filter_function[filter_value]}
    }

In [8]:
@app.callback(
    Output('map-id', 'children'),
    Input('dashGrid', 'selectedRows'))
def output_selected_rows(selected_rows):
    if not selected_rows:
        lat = df.iloc[0]['location_lat']
        lon = df.iloc[0]['location_long']
        animal_id = df.iloc[0]['animal_id']
        return[
            dl.Map(style={'width':'540px', 'height':'500px'},
                center = [lat, lon], 
                zoom=10, 
                children=[
                    dl.TileLayer(id='base-layer-id'),
                    dl.Marker(position=[lat, lon], 
                              icon={
                                'iconUrl': 'https://i.ibb.co/99mLf2b/pin2.png',
                                'iconSize': [32, 32],
                                'iconAnchor': [16, 32],
                                'popupAnchor': [0, -32],
                                },
                              children=[dl.Tooltip(f"Animal ID: {animal_id}")])
                ])
        ]
    
    else:
        selected_data = selected_rows[0]
        lat = selected_data['location_lat']
        lon = selected_data['location_long']
        animal_id = selected_data['animal_id']
        return[
            dl.Map(style={'width':'540px', 'height':'500px'},
                center = [lat, lon], 
                zoom=10, 
                children=[
                    dl.TileLayer(id='base-layer-id'),
                    dl.Marker(position=[lat, lon], 
                              icon={
                                'iconUrl': 'https://i.ibb.co/99mLf2b/pin2.png',
                                'iconSize': [32, 32],
                                'iconAnchor': [16, 32],
                                'popupAnchor': [0, -32],
                                },
                              children=[dl.Tooltip(f"Animal ID: {animal_id}")])
                ])
        ]

In [9]:
if __name__ == "__main__":
    app.run_server(debug=True)